In [ ]:
import pandas as pd
import statistics as stats
import numpy as np

pattern_range = 10
max_trend_points = 150
current_index = 0
x_axis = 'TEST_PERIOD_FORMATTED'
y_axis = 'ATT_TP_UL_MV'
trend_type = "TREND_TYPE"
file_name = 'amCharts.csv'

In [ ]:
data = pd.read_csv(file_name)
df = pd.DataFrame()
df[x_axis] = pd.to_datetime(data[x_axis])
df[y_axis] = data[y_axis]
df = df.dropna().reset_index(drop=True)
df[trend_type] = np.nan
df.head()

,TEST_PERIOD_FORMATTED,ATT_TP_UL_MV,TREND_TYPE
0,2019-07-02,9.15,NaN
1,2019-07-03,9.26,NaN
2,2019-07-04,8.99,NaN
3,2019-07-05,8.97,NaN
4,2019-07-06,9.13,NaN


In [ ]:
current_trend = [] # list of datapoints part of current trend

# for displaying on graph
anomalies = pd.DataFrame(index=range(df.shape[0]),columns=[x_axis, y_axis])
anomalies[x_axis] = df[x_axis]
shifts = pd.DataFrame(index=range(df.shape[0]),columns=[x_axis, y_axis])
shifts[x_axis] = df[x_axis]
trends = pd.DataFrame(index=range(df.shape[0]),columns=[x_axis, y_axis])
trends[x_axis] = df[x_axis]

# builds starting trend from pattern_range number of datapoints
for current_index in range(pattern_range):
  current_trend.append(df[y_axis][current_index])

In [ ]:
for current_index in range(pattern_range - 1, len(df[y_axis])):
  datapoint = df[y_axis][current_index]
  old_mean = stats.mean(current_trend)
  strdev = stats.stdev(current_trend, old_mean)

  # for trends data points on graph
  if (current_index + 1) % (pattern_range/2) == 0:
    trends.iat[current_index, 1] = old_mean
  
  # keeps the current trend list with current points, protects from over-saturation
  if len(current_trend) >= max_trend_points:
    current_trend.pop(0)

  # check if new datapoint is anomaly / trend (3 stdev out)
  if datapoint >= strdev * 3 + old_mean or datapoint <= old_mean - strdev * 3:
    temp_trend = []
    for i in range(pattern_range):
      if (current_index + i < len(df[y_axis])):
        temp_trend.append(df[y_axis][current_index + i])
    new_mean = stats.mean(temp_trend)

    if len(temp_trend) == pattern_range and (new_mean >= strdev * 3 + old_mean or new_mean <= old_mean - strdev * 3): # trend shift
      print("index: " + str(current_index) + " shift: " + str(df[x_axis][current_index]))
      if old_mean < new_mean:
        df[trend_type][current_index] = "shift up"
      elif old_mean > new_mean:
        df[trend_type][current_index] = "shift down"

      shifts.iat[current_index, 1] = datapoint
      current_trend.clear()
      current_trend.extend(temp_trend)
      current_index += pattern_range
    else: # anomaly
      print("index: " + str(current_index) + " anomaly: " + str(df[x_axis][current_index]))
      df[trend_type][current_index] = "anomaly"
      anomalies.iat[current_index, 1] = datapoint
  else:
    current_trend.append(datapoint)

index: 56 shift: 2019-08-27 00:00:00
index: 81 anomaly: 2019-09-21 00:00:00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


index: 255 shift: 2020-03-17 00:00:00
index: 308 anomaly: 2020-05-09 00:00:00
index: 309 shift: 2020-05-10 00:00:00
index: 344 anomaly: 2020-06-14 00:00:00
index: 366 shift: 2020-07-06 00:00:00
index: 388 shift: 2020-07-28 00:00:00
index: 472 shift: 2020-10-23 00:00:00
index: 494 shift: 2020-11-14 00:00:00
index: 506 anomaly: 2020-11-26 00:00:00
index: 535 anomaly: 2020-12-25 00:00:00
index: 586 anomaly: 2021-02-15 00:00:00
index: 587 anomaly: 2021-02-16 00:00:00
index: 588 anomaly: 2021-02-17 00:00:00
index: 589 anomaly: 2021-02-18 00:00:00
index: 803 shift: 2021-09-25 00:00:00
index: 848 anomaly: 2021-11-09 00:00:00
index: 1044 shift: 2022-05-24 00:00:00


In [ ]:
def returnName(mean, first_mean, leniency):
  if mean + mean * leniency >= first_mean and mean - mean * leniency <= first_mean:
    return "consolidated"
  elif mean > first_mean:
    return "up"
  elif mean < first_mean:
    return "down"

# trend identification
first_mean = df[y_axis][0]
i = pattern_range/2 - 1
leniency = 0.001 # leniency for considering means to be equal, 0.1%
name = ""
while i + pattern_range/2 <= len(trends[y_axis]):
  mean = trends[y_axis][i]
  name = returnName(mean, first_mean, leniency)
  j = 0
  if not pd.isnull(mean):
    j = i - (pattern_range/2 - 1)
  else:
    i += pattern_range/2
    mean = trends[y_axis][i]
    name = returnName(mean, first_mean, leniency)
  while j <= i:
    val = df[trend_type][j]
    if pd.isnull(val):
      df[trend_type][j] = name
    j += 1

  first_mean = mean
  i += pattern_range/2

# trend identification for last values
if j < len(trends[y_axis]):
  mean = df[y_axis][len(df[y_axis]) - 1]
  name = returnName(mean, first_mean, leniency)
  while j < len(trends[y_axis]):
    val = df[trend_type][j]
    if pd.isnull(val):
      df[trend_type][j] = name
    j += 1

In [ ]:
# 179 for Jan 2020 consolidated data, 500 for Jan 2021 downards trend
scale = 0
for i in range(len(df[y_axis])):
  print(str(df[x_axis][i + scale]) + " type: " + str(df[trend_type][i + scale]))

In [ ]:
from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import math

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.TEST_PERIOD_FORMATTED, y=df.ATT_TP_UL_MV, 
                         mode='lines', name=y_axis))

fig.add_trace(go.Scatter(x=anomalies.TEST_PERIOD_FORMATTED, y=anomalies.ATT_TP_UL_MV,
                         mode='markers', name='Anomaly [less than 10 points 3 sd above]'))

fig.add_trace(go.Scatter(x=shifts.TEST_PERIOD_FORMATTED, y=shifts.ATT_TP_UL_MV,
                         mode='markers', name='Shift [more than 9 points 3 sd above]'))

fig.add_trace(go.Scatter(x=trends.TEST_PERIOD_FORMATTED, y=trends.ATT_TP_UL_MV,
                         mode='markers', name='Trend [follows mean]'))

fig.update_layout(showlegend=True)
fig.show()